# 💾 📊 Fund Data - Ranking and returns 📊 💾

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
!pip install seaborn
import seaborn as sns


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
myCSVfile = ("merged.csv")
mergedDF = pd.read_csv(myCSVfile)
mergedDF.head()

,Unnamed: 0,Month of Date,Year of Date,Month Number,ETF,Sponsor,Sponsor Category,CIFSC,Fund Name,Net Assets,Gross Sales,Redemptions
0,0,January 2016,2016,1,N,AGF,Independents,U.S. Small/Mid Cap Equity,AGF U.S. Small-Mid Cap Fund,183094000.0,937000.0,2397000.0
1,1,February 2016,2016,2,N,AGF,Independents,U.S. Small/Mid Cap Equity,AGF U.S. Small-Mid Cap Fund,172025000.0,812000.0,1825000.0
2,2,March 2016,2016,3,N,AGF,Independents,U.S. Small/Mid Cap Equity,AGF U.S. Small-Mid Cap Fund,172097000.0,604000.0,1880000.0
3,3,April 2016,2016,4,N,AGF,Independents,U.S. Small/Mid Cap Equity,AGF U.S. Small-Mid Cap Fund,168156000.0,273310.0,1843800.0
4,4,May 2016,2016,5,N,AGF,Independents,U.S. Small/Mid Cap Equity,AGF U.S. Small-Mid Cap Fund,177464000.0,516000.0,1638000.0


In [3]:
# to get the monthly returns, you have to do "Net Asset_(i+1),f" / ("Net Asset_i,f" + ("Gross Sales_(i+1),f - Redemptions_(i+1),f") )
# Add a column with the consolidated CIFSC
import duckdb

# Register both dataframes
duckdb.register("mergedDF", mergedDF)


query = """
SELECT
  "Fund Name",
  "Month of Date",
  "Year of Date",
  "Month Number",
  "ETF",
  "Sponsor",
  "Sponsor Category",
  "CIFSC",
  "Fund Name",
  "Net Assets",
  "Gross Sales",
  "Redemptions",
  CASE
    WHEN LAG("Net Assets") OVER (PARTITION BY "Fund Name" ORDER BY "Month Number") IS NULL THEN 0
    ELSE "Net Assets" / (LAG("Net Assets") OVER (PARTITION BY "Fund Name" ORDER BY "Month Number") 
                          + ("Gross Sales" - "Redemptions"))
  END AS monthly_return
FROM mergedDF;
"""

monthly_returns = duckdb.query(query).to_df()
print(monthly_returns.head(5))


                          Fund Name  Month of Date  Year of Date  \
0  1832 AM Canadian Preferred Share   January 2016          2016   
1  1832 AM Canadian Preferred Share  February 2016          2016   
2  1832 AM Canadian Preferred Share     March 2016          2016   
3  1832 AM Canadian Preferred Share     April 2016          2016   
4  1832 AM Canadian Preferred Share       May 2016          2016   

   Month Number ETF Sponsor Sponsor Category CIFSC  \
0             1   N  Scotia            Banks  None   
1             2   N  Scotia            Banks  None   
2             3   N  Scotia            Banks  None   
3             4   N  Scotia            Banks  None   
4             5   N  Scotia            Banks  None   

                        Fund Name_1  Net Assets  Gross Sales  Redemptions  \
0  1832 AM Canadian Preferred Share         0.0          0.0          0.0   
1  1832 AM Canadian Preferred Share         0.0          0.0          0.0   
2  1832 AM Canadian Preferred Share